<a href="https://colab.research.google.com/github/rams1987/Essay_Scoring/blob/main/essay_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: install kaggle

!pip install kaggle


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ramasamyseenivasagan","key":"f41bb0714d3baef12d51ba89162604ef"}'}

In [3]:
# prompt: create a kaggle folder

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle competitions download -c learning-agency-lab-automated-essay-scoring-2

 50% 6.00M/11.9M [00:00<00:00, 61.0MB/s]
100% 11.9M/11.9M [00:00<00:00, 96.6MB/s]


In [5]:
# prompt: unzip the downloaded file

!unzip learning-agency-lab-automated-essay-scoring-2.zip


Archive:  learning-agency-lab-automated-essay-scoring-2.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [6]:
import os
import numpy as np
import pandas as pd
import spacy

In [7]:
essay_df = pd.read_csv('train.csv')

essay_df.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [10]:
# prompt: get the sentence count, word count, average sentence length, vocabulary size or unique words in the essay and sentiment of the essagy

# Sentence count
essay_df['sentence_count'] = essay_df['full_text'].apply(lambda x: len(x.split('.')))

# Word count
essay_df['word_count'] = essay_df['full_text'].apply(lambda x: len(x.split()))

# Average sentence length
essay_df['average_sentence_length'] = essay_df['word_count'] / essay_df['sentence_count']

# Vocabulary size or unique words
essay_df['unique_words'] = essay_df['full_text'].apply(lambda x: len(set(x.split())))

# Print the results
essay_df.head()


,essay_id,full_text,score,sentence_count,word_count,average_sentence_length,unique_words
0,000d118,Many people have car where they live. The thin...,3,14,498,35.571429,253
1,000fe60,I am a scientist at NASA that is discussing th...,3,20,332,16.600000,177
2,001ab80,People always wish they had the same technolog...,4,25,550,22.000000,253
3,001bdc0,"We all heard about Venus, the planet without a...",4,24,451,18.791667,251
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3,16,373,23.312500,162


In [11]:
# prompt: Using dataframe essay_df: add a column with sentiment

from textblob import TextBlob
essay_df['sentiment'] = essay_df['full_text'].apply(lambda x: TextBlob(x).sentiment.polarity)


In [12]:
essay_df.head()

,essay_id,full_text,score,sentence_count,word_count,average_sentence_length,unique_words,sentiment
0,000d118,Many people have car where they live. The thin...,3,14,498,35.571429,253,0.179020
1,000fe60,I am a scientist at NASA that is discussing th...,3,20,332,16.600000,177,0.101786
2,001ab80,People always wish they had the same technolog...,4,25,550,22.000000,253,0.168435
3,001bdc0,"We all heard about Venus, the planet without a...",4,24,451,18.791667,251,0.094291
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3,16,373,23.312500,162,0.170952


In [21]:
# prompt: Using dataframe essay_df: train a model to predict score of the essay

from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(essay_df[['sentence_count', 'word_count', 'average_sentence_length', 'unique_words', 'sentiment']], essay_df['score'], test_size=0.25)

# Train a linear regression model
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the

LinearRegression()

In [22]:
test_df.head()

,essay_id,full_text,sentence_count,word_count,average_sentence_length,unique_words,sentiment
0,000d118,Many people have car where they live. The thin...,14,498,35.571429,253,0.179020
1,000fe60,I am a scientist at NASA that is discussing th...,20,332,16.600000,177,0.101786
2,001ab80,People always wish they had the same technolog...,25,550,22.000000,253,0.168435


In [20]:
# prompt: test the model against the test.csv

import pandas as pd
test_df = pd.read_csv('test.csv')

# Calculate the features for the test set
test_df['sentence_count'] = test_df['full_text'].apply(lambda x: len(x.split('.')))
test_df['word_count'] = test_df['full_text'].apply(lambda x: len(x.split()))
test_df['average_sentence_length'] = test_df['word_count'] / test_df['sentence_count']
test_df['unique_words'] = test_df['full_text'].apply(lambda x: len(set(x.split())))
test_df['sentiment'] = test_df['full_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Make predictions on the test set
predictions = model.predict(test_df[['sentence_count', 'word_count', 'average_sentence_length', 'unique_words', 'sentiment']])

# Evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(test_df['score'], predictions)
print('Mean squared error:', mse)



KeyError: 'score'

In [ ]:
# prompt: train a model to predict score of the essay

from sklearn.linear_model import LinearRegression

X = essay_df[['sentence_count', 'word_count']]
y = essay_df['score']

model = LinearRegression().fit(X, y)


In [ ]:
# prompt: test the model against the test.csv

import pandas as pd
test_df = pd.read_csv('test.csv')
test_df["sentence_count"] = test_df["full_text"].apply(lambda text: len(list(nlp(text).sents)))
test_df["word_count"] = test_df["full_text"].apply(lambda text: len(text.split()))

X_test = test_df[['sentence_count', 'word_count']]
y_pred = model.predict(X_test)

print(y_pred)


[3.44165299 2.79653307 3.79060125]


In [ ]:
test_df['score'] = y_pred
test_df.head()

,essay_id,full_text,sentence_count,word_count,score
0,000d118,Many people have car where they live. The thin...,13,498,3.441653
1,000fe60,I am a scientist at NASA that is discussing th...,20,332,2.796533
2,001ab80,People always wish they had the same technolog...,25,550,3.790601
